<a href="https://colab.research.google.com/github/deburg0/DLwPython/blob/main/Reimplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reimplementing our first example from scratch

A dense layer applies the transformation $y = a(W x + b)$

In [1]:
# keras.ops is where the tensor operations are defined
import keras
from keras import ops

In [2]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation=None):
        self.activation = activation

        self.W = keras.Variable(
            shape=(input_size, output_size),
            initializer="uniform"
        )

        self.b = keras.Variable(shape=(output_size,), initializer="zeros")

# applies the forward pass
    def __call__(self, inputs):
        x = ops.matmul(inputs, self.W) + self.b
        if self.activation is not None:
            x = self.activation(x)
        return x
    @property
# the convenience method for retrieving the layers weights
    def weights(self):
        return [self.W, self.b]


### A simple Sequential class

Sequential class allows us to chain layers together and exposes a call method to call the layers in order.

In [4]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [5]:
model = NaiveSequential(
    [
        NaiveDense(input_size=28 * 28, output_size=512, activation=ops.relu),
        NaiveDense(input_size=512, output_size=10, activation=ops.softmax)

    ]
)
assert len(model.weights) == 4

#### A batch generator

In [6]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

## Running one training step

#### The weight update step

In [9]:
from keras import optimizers
optimizer = optimizers.SGD(learning_rate=1e-3)
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [14]:
import tensorflow as tf
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        loss = ops.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = ops.mean(loss)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [15]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

In [16]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 2.30
loss at batch 100: 2.27
loss at batch 200: 2.22
loss at batch 300: 2.20
loss at batch 400: 2.16
Epoch 1
loss at batch 0: 2.13
loss at batch 100: 2.11
loss at batch 200: 2.04
loss at batch 300: 2.03
loss at batch 400: 1.99
Epoch 2
loss at batch 0: 1.94
loss at batch 100: 1.94
loss at batch 200: 1.85
loss at batch 300: 1.85
loss at batch 400: 1.81
Epoch 3
loss at batch 0: 1.74
loss at batch 100: 1.77
loss at batch 200: 1.64
loss at batch 300: 1.66
loss at batch 400: 1.63
Epoch 4
loss at batch 0: 1.54
loss at batch 100: 1.58
loss at batch 200: 1.44
loss at batch 300: 1.46
loss at batch 400: 1.46
Epoch 5
loss at batch 0: 1.36
loss at batch 100: 1.40
loss at batch 200: 1.26
loss at batch 300: 1.29
loss at batch 400: 1.30
Epoch 6
loss at batch 0: 1.19
loss at batch 100: 1.25
loss at batch 200: 1.10
loss at batch 300: 1.14
loss at batch 400: 1.17
Epoch 7
loss at batch 0: 1.05
loss at batch 100: 1.11
loss at batch 200: 0.97
loss at batch 300: 1.02
loss at batch 40

In [18]:
predictions = model(test_images)
predictions = ops.argmax(predictions, axis=1)
matches = ops.equal(predictions, test_labels)
accuracy = ops.mean(matches)
print(f"accuracy: {accuracy:.2f}")

accuracy: 0.84


In [19]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 0.79
loss at batch 100: 0.84
loss at batch 200: 0.72
loss at batch 300: 0.79
loss at batch 400: 0.86
Epoch 1
loss at batch 0: 0.73
loss at batch 100: 0.78
loss at batch 200: 0.67
loss at batch 300: 0.74
loss at batch 400: 0.81
Epoch 2
loss at batch 0: 0.69
loss at batch 100: 0.73
loss at batch 200: 0.62
loss at batch 300: 0.69
loss at batch 400: 0.78
Epoch 3
loss at batch 0: 0.65
loss at batch 100: 0.69
loss at batch 200: 0.59
loss at batch 300: 0.66
loss at batch 400: 0.74
Epoch 4
loss at batch 0: 0.62
loss at batch 100: 0.65
loss at batch 200: 0.56
loss at batch 300: 0.63
loss at batch 400: 0.71
Epoch 5
loss at batch 0: 0.59
loss at batch 100: 0.62
loss at batch 200: 0.53
loss at batch 300: 0.60
loss at batch 400: 0.69
Epoch 6
loss at batch 0: 0.57
loss at batch 100: 0.59
loss at batch 200: 0.51
loss at batch 300: 0.58
loss at batch 400: 0.67
Epoch 7
loss at batch 0: 0.55
loss at batch 100: 0.57
loss at batch 200: 0.48
loss at batch 300: 0.56
loss at batch 40

In [20]:
predictions = model(test_images)
predictions = ops.argmax(predictions, axis=1)
matches = ops.equal(predictions, test_labels)
accuracy = ops.mean(matches)
print(f"accuracy: {accuracy:.2f}")

accuracy: 0.87
